In [44]:
import re
from os import listdir
from os.path import isfile, join
import random
import json

In [49]:
def split_script(src, dest, block_count, test_count):
    with open(src, "r") as f:
        lines = f.readlines()
        line_count = len(lines)
        lines_per_block = line_count // block_count
        blocks = random.sample(range(0, block_count), test_count)
        data = {"train":""}
        test_itr = 1
        for i in range(block_count):
            start_index = i*lines_per_block
            end_index = start_index + lines_per_block
            split_index = (start_index + end_index) // 2
            
            if i in blocks:
                data["test"+str(test_itr)+"_input"] = ''.join(lines[start_index:split_index])
                data["test"+str(test_itr)+"_true"]  = ''.join(lines[split_index+1:end_index])
                test_itr += 1
            else:
                data["train"] += ''.join(lines[start_index:split_index])
        print(data.keys())
        dest = dest[:-4] + ".json"
        with open(dest, 'w+') as outfile:
            json.dump(data, outfile)
            

def generate_tests_entire_dir(srcpath, destpath):
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            split_script(src, dest, 20, 3)
            break
    
def parse_script(src, dest):
    with open(src, "r") as f:
        parsed = ""
        added_new = True # Used for ignoring consequtive empty lines
        for line in f:
            temp = line.strip(" ")
            if temp != "\n":
                temp = temp.strip("\n")
                if (re.search('[a-zA-Z]', temp)): # Ignore page number lines
                    parsed += line.strip() + " "
                    added_new = False
            else:
                if not added_new:
                    parsed += "\n"
                    added_new = True
                continue
    with open(dest, "w") as f:
        f.write(parsed)

def process_entire_dir(srcpath, destpath):
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            parse_script(src, dest)
    print("Processed: {} scripts".format(len(listdir(srcpath))))

In [ ]:
process_entire_dir("./raw_data/", "./parsed_data/")

In [50]:
generate_tests_entire_dir("./parsed_data/", "./test_cases2/")

dict_keys(['train', 'test1_input', 'test1_true', 'test2_input', 'test2_true', 'test3_input', 'test3_true'])
